In [39]:

import numpy as np
from scipy.optimize import fsolve 

import matplotlib.pyplot as plt

In [40]:
def electroneut(N_i, K_i, C_i, z_x, X):
    electroneut = (N_i + K_i - C_i) + z_x * X

    return electroneut

def osmoticbal(N_i, K_i, C_i, X, N_b, K_b, C_b, N_a, K_a, C_a):
    osmoticbal = N_b + K_b + C_b - 2*(N_i + K_i + C_i + (X)) + N_a + K_a + C_a

    return osmoticbal

def NaKATPase(N_i, K_na, P, K_a, K_k, K_i, N_a):
    NaKATPase = P * ((N_i / (K_na + N_i))**3) * (K_a / (K_k + K_a))**2

    return NaKATPase

def Cotransporter(M, N_i, K_i, C_i, N_a, K_a, C_a):
    Cotransporter =  M * np.log((N_i * K_i * (C_i)**2)/(N_a * K_a * (C_a)**2))

    return Cotransporter

def GHKflux(z, Ps_k, Vsm, ns_k, nm_k):
    frt = 37.435883507802615 # Using F, R and T from Mariia's paper
    GHKflux = z * Ps_k * (frt) * Vsm * ((ns_k - nm_k * np.exp((-z * frt * Vsm)))/(1 - np.exp((-z * frt * Vsm))))

    return GHKflux




In [41]:
# Standard parameters
P = 10**(-6) # Non-dimensional
z_x = 1.5 # Non-dimensional
M = 3 * 10**(-6)
z_k = 1 # Non-dimensional
z_c = -1 # Non-dimensional

#Permeabilities
Pa_K = 5*10**(-7)
Pb_K = 1.5*10**(-7)
Ptj_K = 10**(-6)
Pa_C = 0
Pb_C = 0.8*10**(-8)
Ptj_C = 0

#Concentration, and derived values
N_a = 0.120
K_a = 0.005 
C_a = 0.125
N_b = 0.120
K_b =  0.005
C_b = 0.125
K_na = 0.02692677070828331 # 0.2 * ((1 + K_i)/8.33)
K_k = 0.006054054054054055 # 0.1 * ((1 + N_a)/18.5)

# Variables we want are: N_i, K_i, C_i, X, V

In [42]:
#Defining the whole system

def sys(vars, pars = None):
    N_i, K_i, C_i, X, V = vars


    eq1 = -3 * NaKATPase(N_i, K_na, P, K_a, K_k, K_i, N_a) + Cotransporter(M, N_i, K_i, C_i, N_a, K_a, C_a)
    eq2 = -1 * GHKflux(z_k, Pa_K, V, K_i, K_a) - GHKflux(z_k, Pb_K, V, K_i, K_b) + 2 * NaKATPase(N_i, K_na, P, K_a, K_k, K_i, N_a) + Cotransporter(M, N_i, K_i, C_i, N_a, K_a, C_a)
    eq3 = 2 * Cotransporter(M, N_i, K_i, C_i, N_a, K_a, C_a) - GHKflux(z_c, Pb_C, V, C_i, C_b)
    eq4 = electroneut(N_i, K_i, C_i, z_x, X)
    eq5 = osmoticbal(N_i, K_i, C_i, X, N_b, K_b, C_b, N_a, K_a, C_a)

    return [eq1, eq2, eq3, eq4, eq5]

#Initial guess
initial_guess = [0.018, 0.120, 0.05, 0.1, 0.014]

#Solving the system
solution = fsolve(sys,initial_guess)

print(f"Solution: $Na_i$ = {solution[0]}, $K_i$ = {solution[1]}, $Cl_i$ = {solution[2]}, V = {solution[3]}, X = {solution[4]}")

Solution: $Na_i$ = 0.018, $K_i$ = 0.12, $Cl_i$ = 0.05, V = 0.1, X = 0.014


/var/folders/kg/6qgzh1c962vg2xt2n9swng0h0000gn/T/ipykernel_53410/4182648224.py:17: RuntimeWarning: invalid value encountered in log
  Cotransporter =  M * np.log((N_i * K_i * (C_i)**2)/(N_a * K_a * (C_a)**2))
/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [ ]:
# Plot with fixed values to find error

# Then unassume that voltage is same at bottom and top and solve for voltage at top and bottom.